This is the code which will help me harness the power of the GPU's in my macOS

In [1]:
import tensorflow as tf

tf.debugging.set_log_device_placement(True)

This is the code that will help me get the photos in the `sed_test` and `sed_train` folders inside the dataset.

In [9]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

data_dir = '/Users/adityaamehra/Desktop/ML projects/Mosaic/archive'

# Image preprocessing and data augmentation
image_size = (150, 150)  # Resize images to 150x150
batch_size = 32

# Data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Rescale for validation and test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Load data
train_data = train_datagen.flow_from_directory(
    os.path.join(data_dir, 'train'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)
test_data = test_datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Print class indices
print("Class indices:", train_data.class_indices)


Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Class indices: {'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}


This is the code that will be implementing logistic regression for getting the predictions done; we have used the sci-kit-learn library for implementing the logistic regression,

I have made a `X_train` array and a `Y_train` array , which will have the training images in them and will just use the sci-kit-learn library to implement the logistic regression and get the tests done using the `X_test` and `Y_test` arrays.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Prepare training data
X_train = np.array([img for img, _ in train_data])
y_train = np.array(train_data.classes)

# Flatten the training images
X_train_flattened = X_train.reshape(X_train.shape[0], -1)

# Normalize data to scale pixel values between 0 and 1,  for better logistic regression implementation
X_train_flattened = X_train_flattened / 255.0

# Train Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_flattened, y_train)

# Evaluate on test set with batch processing
X_test = np.array([img for img, _ in test_data])
y_test = np.array(test_data.classes)

# Flatten the test images
X_test_flattened = X_test.reshape(X_test.shape[0], -1) / 255.0

# Batch size for processing
batch_size = 64
num_batches = int(np.ceil(X_test_flattened.shape[0] / batch_size))

# Initialize predictions list
y_pred = []

print("Starting batch predictions...")

# Process test set in batches
for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min(start_idx + batch_size, X_test_flattened.shape[0])
    batch_X = X_test_flattened[start_idx:end_idx]
    
    # Predict for the current batch
    batch_pred = log_reg.predict(batch_X)
    y_pred.extend(batch_pred)

    # Progress update
    print(f"Processed batch {batch_idx + 1}/{num_batches}")

# Convert predictions to a NumPy array
y_pred = np.array(y_pred)

# Print evaluation metrics
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Made a simple ANN (artificial neural network) using the `keras` package in `tensorflow` and using `Adam` as the optimiser

In [ ]:
# Build a simple ANN
ann_model = Sequential([
    Flatten(input_shape=(150, 150, 3)),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(6, activation='softmax')  # 6 classes for softmax
])

ann_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the ANN model
ann_model.fit(train_data, epochs=10, validation_data=test_data)

# Evaluate on test data
test_loss, test_acc = ann_model.evaluate(test_data)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op FloorMod in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0


/opt/anaconda3/envs/mosaic/lib/python3.9/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-02-03 13:38:08.310489: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-02-03 13:38:08.310515: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-02-03 13:38:08.310524: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-02-03 13:38:08.310554: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-03 13:38:08.310564: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhos

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomUniformV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in

2025-02-03 13:38:08.557719: I tensorflow/core/common_runtime/placer.cc:125] input: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-03 13:38:08.557730: I tensorflow/core/common_runtime/placer.cc:125] _EagerConst: (_EagerConst): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-03 13:38:08.557732: I tensorflow/core/common_runtime/placer.cc:125] output_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-03 13:38:08.559900: I tensorflow/core/common_runtime/placer.cc:125] seed: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2025-02-03 13:38:08.559908: I tensorflow/core/common_runtime/placer.cc:125] StatelessRandomGetKeyCounter: (StatelessRandomGetKeyCounter): /job:localhost/replica:0/task:0/device:CPU:0
2025-02-03 13:38:08.559914: I tensorflow/core/common_runtime/placer.cc:125] key_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:CPU:0
2025-02-03 13:38:08.559916: I tensorflow/core/common_runtime/placer.cc:125] counter_RetVal: (_Retval): /jo

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op FloorMod in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StatelessRandomUniformV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /jo

...

dense_1_kernel_velocity/Initializer/zeros/shape_as_tensor: (Const): /job:localhost/replica:0/task:0/device:GPU:0
adam/sequential_dense_1_kernel_velocity/Initializer/zeros/Const: (Const): /job:localhost/replica:0/task:0/device:GPU:0
adam/sequential_dense_1_bias_momentum/Initializer/zeros: (Const): /job:localhost/replica:0/task:0/device:GPU:0
adam/sequential_dense_1_bias_velocity/Initializer/zeros: (Const): /job:localhost/replica:0/task:0/device:GPU:0
adam/sequential_dense_2_kernel_momentum/Initializer/zeros/shape_as_tensor: (Const): /job:localhost/replica:0/task:0/device:GPU:0
adam/sequential_dense_2_kernel_momentum/Initializer/zeros/Const: (Const): /job:localhost/replica:0/task:0/device:GPU:0
adam/sequential_dense_2_kernel_velocity/Initializer/zeros/shape_as_tensor: (Const): /job:localhost/replica:0/task:0/device:GPU:0
adam/sequential_dense_2_kernel_velocity/Initializer/zeros/Const: (Const): /job:localhost/replica:0/task:0/device:GPU:0
adam/sequential_dense_2_bias_momentum/Initializer/

2025-02-03 13:38:09.358396: I tensorflow/core/common_runtime/placer.cc:125] iterator: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2025-02-03 13:38:09.358415: I tensorflow/core/common_runtime/placer.cc:125] unknown: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-03 13:38:09.358417: I tensorflow/core/common_runtime/placer.cc:125] unknown_0: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-03 13:38:09.358419: I tensorflow/core/common_runtime/placer.cc:125] unknown_1: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-03 13:38:09.358421: I tensorflow/core/common_runtime/placer.cc:125] unknown_2: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-03 13:38:09.358422: I tensorflow/core/common_runtime/placer.cc:125] unknown_3: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-03 13:38:09.358423: I tensorflow/core/common_runtime/placer.cc:125] unknown_4: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-03 13:38:09.358

Executing op OptionalHasValue in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op OptionalGetValue in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Range in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mean in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Range in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mean in device /job:localhost/replica:0/task:0/device:GPU:0
  1/439 ━━━━━━━━━━━━━━━━━━━━ 14:06 2s/step - accuracy: 0.0938 - loss: 2.6924Executing op __inference_multi_step_on_iterator_1321 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op OptionalHasValue in

2025-02-03 13:38:10.999435: I tensorflow/core/common_runtime/placer.cc:125] optional: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-03 13:38:10.999449: I tensorflow/core/common_runtime/placer.cc:125] OptionalHasValue: (OptionalHasValue): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-03 13:38:10.999451: I tensorflow/core/common_runtime/placer.cc:125] has__value_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-03 13:38:11.012062: I tensorflow/core/common_runtime/placer.cc:125] optional: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-03 13:38:11.012072: I tensorflow/core/common_runtime/placer.cc:125] OptionalGetValue: (OptionalGetValue): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-03 13:38:11.012074: I tensorflow/core/common_runtime/placer.cc:125] components_0_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
2025-02-03 13:38:11.012076: I tensorflow/core/common_runtime/placer.cc:125] components_1_RetVal: (_R

Executing op OptionalHasValue in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op OptionalGetValue in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Range in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mean in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Range in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mean in device /job:localhost/replica:0/task:0/device:GPU:0
  4/439 ━━━━━━━━━━━━━━━━━━━━ 33s 76ms/step - accuracy: 0.1094 - loss: 112.2299Executing op __inference_multi_step_on_iterator_1321 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op OptionalHasValue in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op OptionalGetValue in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Range i

Made a simple CNN (convolutional neural network) using the `keras` package in `tensorflow` and using `Adam` as the optimiser

In [4]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D

# Build a CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(6, activation='softmax')
])

cnn_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model
cnn_model.fit(train_data, epochs=10, validation_data=test_data)

# Evaluate on test data
test_loss, test_acc = cnn_model.evaluate(test_data)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

NameError: name 'Sequential' is not defined

Implemented transfer learning with a pre-trained model using VGG16

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load pre-trained VGG16 model without the top classification layers
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze convolutional layers
for layer in vgg_model.layers:
    layer.trainable = False

# Build the model
transfer_model = Sequential([
    vgg_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(6, activation='softmax')  # Assuming 6 classes
])

# Compile the model
transfer_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Data generators
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
val_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Train the transfer learning model, this will divert the computations to GPU for multithreaded computations
with tf.device('/GPU:0'):
    transfer_model.fit(
        train_data,
        epochs=10,
        validation_data=test_data)

# Evaluate on test data
test_loss, test_acc = transfer_model.evaluate(test_data)``
print(f"Test Accuracy: {test_acc * 100:.2f}%")